# Import related functions

In [1]:
import math
import cobra
#from script.ECMpy_function import *
import sys
sys.path.append(r'./script/')
from ECMpy_function import *

# Directly determining targets based on enzyme abundance

In [2]:
ecModel_file="./model/iML1515_irr_enz_constraint_adj.json"
obj='EX_lys__L_e' # EX_lys_L_e
substrate = 'EX_glc__D_e'
substrate_con=5
biomass_id = 'BIOMASS_Ec_iML1515_core_75p37M'
biomass_min=0.1
biomass_max=0.2

enzcost = pd.DataFrame()
fluxes_outfile = './analysis/ECMpy_solution_%s_%.2g_pfba.csv'%(obj,substrate_con)

enz_model=get_enzyme_constraint_model(ecModel_file)
enz_model.reactions.get_by_id(substrate).bounds=(-substrate_con,0)#EX_glc_e EX_fru_e EX_sucr_e EX_inost_e EX_lac_D_e EX_ac_e
enz_model.reactions.get_by_id('%s_reverse'%substrate).bounds=(0,0)
enz_model.reactions.get_by_id(biomass_id).bounds=(biomass_min,biomass_min)
enz_model.objective=obj
enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)

enzcost = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_file)
enzcost = enzcost.sort_values('E', ascending=False)
enzcost.head(10)# TOP10

,fluxes,kcat_MW,E,equ,ec-code
ATPS4rpp_num2,17.224995,285.026853,0.060433,adp_c + 4.0 h_p + pi_c --> atp_c + h2o_c + 3.0...,3.6.3.14
NADH16pp,9.800096,173.200546,0.056582,4.0 h_c + nadh_c + q8_c --> 3.0 h_p + nad_c + ...,1.6.5.3
GLUDy_reverse,6.567265,335.933801,0.019549,akg_c + h_c + nadph_c + nh4_c --> glu__L_c + h...,NaN
RPE,4.214189,293.231245,0.014372,ru5p__D_c --> xu5p__D_c,5.1.3.1
LEUTAi_num2,0.045053,3.307456,0.013622,4mop_c + glu__L_c --> akg_c + leu__L_c,"2.6.1.42,2.6.1.6,2.6.1.67"
PDH,0.773324,72.304633,0.010695,coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...,"1.2.1.-,1.2.1.51,1.2.4.1,1.8.1.4,2.3.1.12"
KARA1_reverse,0.087437,8.489153,0.010300,alac__S_c + h_c + nadph_c --> 23dhmb_c + nadp_c,NaN
PGL,6.437654,1417.869340,0.004540,6pgl_c + h2o_c --> 6pgc_c + h_c,3.1.1.31
ASPK_num3,2.962901,775.664416,0.003820,asp__L_c + atp_c --> 4pasp_c + adp_c,2.7.2.4
ACONTa_num1,0.739241,204.068536,0.003623,cit_c --> acon_C_c + h2o_c,4.2.1.3


#  Determine targets based on the fold changes of enzyme cost 

In [3]:
ecModel_file="./model/iML1515_irr_enz_constraint_adj.json"
obj='EX_lys__L_e' # EX_lys_L_e
substrate = 'EX_glc__D_e'
substrate_con=5
biomass_id = 'BIOMASS_Ec_iML1515_core_75p37M'
biomass_min=0.1
biomass_max=0.2
FC_threshold = 1.5

enzcost_diff_file = './analysis/enzcost_diff.csv'
fluxes_outfile = './analysis/ECMpy_solution_%s_%.2g_pfba.csv'%(obj,substrate_con)

enzcost_diff = get_enz_foldchange(ecModel_file,obj,substrate,substrate_con,biomass_id,biomass_min,biomass_max,FC_threshold,fluxes_outfile, enzcost_diff_file)
enzcost_diff.head(10)

,E_μ0.1,E_μ0.2,log2_foldchange(max/min),log2_foldchange(min/max),type,foldchange(max/min),equ,ec-code
NADH16pp,5.658236e-02,9.921739e-03,-2.511687,2.511687,weaken_target,0.175350,4.0 h_c + nadh_c + q8_c --> 3.0 h_p + nad_c + ...,1.6.5.3
CS,7.584333e-04,2.516846e-03,1.730523,-1.730523,enhance_target,3.318480,accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c,"2.3.3.1,2.3.3.16,2.3.3.3"
MDH,1.187299e-08,4.027180e-08,1.762087,-1.762087,enhance_target,3.391884,mal__L_c + nad_c --> h_c + nadh_c + oaa_c,"1.1.1.299,1.1.1.37"
FUM_num1,3.452021e-05,1.147550e-04,1.733044,-1.733044,enhance_target,3.324284,fum_c + h2o_c --> mal__L_c,4.2.1.2
ICL,1.890199e-06,6.562727e-06,1.795758,-1.795758,enhance_target,3.471977,icit_c --> glx_c + succ_c,4.1.3.1
MALS_num1,2.199071e-04,7.634897e-04,1.795714,-1.795714,enhance_target,3.471873,accoa_c + glx_c + h2o_c --> coa_c + h_c + mal_...,2.3.3.9
SUCDi,1.843953e-07,6.532657e-07,1.824868,-1.824868,enhance_target,3.542746,q8_c + succ_c --> fum_c + q8h2_c,NaN
ACONTb_num1,9.468061e-05,3.141957e-04,1.730523,-1.730523,enhance_target,3.318480,acon_C_c + h2o_c --> icit_c,4.2.1.3
ACONTa_num1,3.622516e-03,1.202125e-02,1.730523,-1.730523,enhance_target,3.318480,cit_c --> acon_C_c + h2o_c,4.2.1.3
FDH4pp_num1,3.839606e-07,2.216872e-06,2.529495,-2.529495,enhance_target,5.773697,for_p + 2.0 h_c + q8_c --> co2_p + h_p + q8h2_c,NaN


# Determine targets based on FESOF

In [4]:
ecModel_file="./model/iML1515_irr_enz_constraint_adj.json"
model=cobra.io.json.load_json_model(ecModel_file)
#model=get_enzyme_constraint_model(ecModel_file)

obj='EX_lys__L_e' 
substrate = 'EX_glc__D_e'
substrate_con=5
biomass_id = 'BIOMASS_Ec_iML1515_core_75p37M'

FSEOF_file = './analysis/FESEOF_gene.csv'
[FSEOFdf_detail,FESEOF_gene] = run_FSEOF(model,substrate,substrate_con,biomass_id,obj,FSEOF_file)
FSEOFdf_detail.head(10)

./script/ECMpy_function.py:2478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  always_down['regulation'] = 'down'
./script/ECMpy_function.py:2481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  always_up['regulation'] = 'up'
./script/ECMpy_function.py:2494: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  FSEOFdf_g['genes'] = FSEOFdf_g['genes'].str.replace

,genes,growth = 0.21,FC growth = 0.21,growth = 0.23,FC growth = 0.23,growth = 0.25,FC growth = 0.25,growth = 0.27,FC growth = 0.27,growth = 0.29,...,growth = 0.36,FC growth = 0.36,growth = 0.38,FC growth = 0.38,WT,GPR,FC_mean,regulation,check,reactions
200,b0929,19.348407,1000.000000,19.767997,1000.000000,20.190026,1000.000000,20.612055,1000.000000,21.034083,...,22.511184,1000.000000,22.933213,1000.000000,0.000000,b0929,554.977126,up,yes,"[2DGLCtex_reverse_num4, ACtex_reverse_num3, CL..."
332,b3612,7.616860,1000.000000,7.629626,1000.000000,7.643124,1000.000000,7.656622,1000.000000,7.670119,...,7.717362,1000.000000,7.730860,1000.000000,0.000000,b3612,548.939736,up,yes,"[PGM_num2, PGM_reverse_num2]"
386,b4090,1.671764,1000.000000,1.591247,1000.000000,1.509999,1000.000000,1.428751,1000.000000,1.347503,...,1.063135,1000.000000,0.981887,1000.000000,0.000000,b4090,546.074433,up,yes,"[RPI_reverse_num2, ALLPI, RPI_num2, ALLPI_reve..."
107,b2838,1.966458,13.576306,1.795233,12.394179,1.623398,11.207841,1.451563,10.021504,1.279728,...,0.678307,4.682987,0.506472,3.496650,0.144845,b2838,5.847894,up,yes,[DAPDC]
389,b2472,1.972293,12.596567,1.801624,11.506543,1.630345,10.412624,1.459066,9.318706,1.287787,...,0.688310,4.396073,0.517031,3.302155,0.156574,b2472,5.484965,up,yes,[SDPDS]
121,b0031,1.972293,12.596567,1.801624,11.506543,1.630345,10.412624,1.459066,9.318706,1.287787,...,0.688310,4.396073,0.517031,3.302155,0.156574,b0031,5.484965,up,yes,[DHDPRy]
108,b3809,1.972293,12.596567,1.801624,11.506543,1.630345,10.412624,1.459066,9.318706,1.287787,...,0.688310,4.396073,0.517031,3.302155,0.156574,b3809,5.484965,up,yes,"[DAPE_reverse, DAPE]"
405,b0166,1.972293,12.596567,1.801624,11.506543,1.630345,10.412624,1.459066,9.318706,1.287787,...,0.688310,4.396073,0.517031,3.302155,0.156574,b0166,5.484965,up,yes,[THDPS]
122,b2478,1.972293,12.596567,1.801624,11.506543,1.630345,10.412624,1.459066,9.318706,1.287787,...,0.688310,4.396073,0.517031,3.302155,0.156574,b2478,5.484965,up,yes,[DHDPS]
298,b0451,6.056771,1.328666,5.916592,1.297915,5.775194,1.266897,5.633797,1.235878,5.492399,...,4.997507,1.096296,4.856109,1.065278,4.558537,b0451,3.177875,up,yes,"[NH4tpp, NH4tpp_reverse]"


In [5]:
FESEOF_gene.head(10)

,genes,regulation,reactions,check
200,b0929,up,"[2DGLCtex_reverse_num4, ACtex_reverse_num3, CL...",yes
332,b3612,up,"[PGM_num2, PGM_reverse_num2]",yes
386,b4090,up,"[RPI_reverse_num2, ALLPI, RPI_num2, ALLPI_reve...",yes
107,b2838,up,[DAPDC],yes
389,b2472,up,[SDPDS],yes
121,b0031,up,[DHDPRy],yes
108,b3809,up,"[DAPE_reverse, DAPE]",yes
405,b0166,up,[THDPS],yes
122,b2478,up,[DHDPS],yes
298,b0451,up,"[NH4tpp, NH4tpp_reverse]",yes
